In [1]:
using Distributions
using Optim

In [42]:
function random_select(n, params, a, x, e; rho = -2)
    #set up params
    lambda = exp(rho)/(1+exp(rho))
    delta = params[1:n]
    beta = params[n+1:2*n] 
    
    #setup payoffs
    u_lower = beta.*x  .+ e
    u_upper = u_lower[:] .+ delta
    
    #set up a
    a_return = zeros(n)
    select = rand(Categorical([lambda, (1-lambda)]))
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
        if (length(a[i])>1) & (select == i)
            println("random select: ", select, " ", i)
            a_return[i] = 1 .- a[i][1]
        end
    end
    
    if length(a[1])>1
        println(select, " ", a)
        println(u_lower, " ", u_upper)
        println(a_return)
    end
    
    return a_return
end


function generate_obs(n, params; verbose=false, select_eq=random_select)
    delta = params[1:n]
    beta = params[n+1:2*n]    
    
    x= rand(Normal(0.0, 1.0),n)
    e = rand(Logistic(),n)
    
    a = []
    a_next = []
    for i=1:n
        append!(a_next,[[]])
        append!(a,[[0,1]])
    end
    eq = false
    u_lower = zeros(n)
    u_upper = zeros(n)
    
    #iterative best responses
    while !eq
        eq = true
        #compute upper and lower bounds on utility...
        u_lower = beta.*x .+ e
        u_upper = u_lower[:]
        #compute correct bounds for each player
        for i=1:n
            for j=1:(n-1)
                j_index = mod(i+j-1,n)+1
                u_upper[i] = u_upper[i] + delta[i]*minimum(a[j_index])
                u_lower[i] = u_lower[i] + delta[i]*maximum(a[j_index])
            end
            
            
            
            #check to see if 0 or 1 is undominated
            if u_lower[i] <= 0
                a_next[i] = append!(a_next[i],0)
            end
                
            if u_upper[i] >=0
                a_next[i] = append!(a_next[i],1)
            end
            if verbose
                println(i," ", a[i]," ", a_next[i]," ", (a_next[i] == a[i]),  " ", eq)
                println(i," ", u_lower[i]," ", u_upper[i])
            end
            eq = eq & (a_next[i] == a[i])
        end
        if verbose
            println("----")
        end
        a =  a_next
        a_next = []
        for i=1:n
            append!(a_next,[[]])
        end
    end
    
    #reshape a correctly/select equilibria if mult
    a_return = select_eq(n, params, a, x, e)
    return a_return,x
end


function gen_data(N;n=3,select_eq=random_select,params= [.6, .7, .8, -3,-3.5,-4])
    Y = zeros(n,N)
    X = zeros(n,N)
    for i=1:N
        y,x = generate_obs(n,params;select_eq=select_eq)
        Y[:,i] = y
        X[:,i] = x
    end
    return Y,X
end



gen_data(10;n=2,params=[-3,-3,.0, .0],select_eq=random_select)

random select: 2 2
2 Any[Any[0, 1], Any[0, 1]]
[1.33916, 1.08031] [-1.66084, -1.91969]
[0.0, 1.0]
random select: 2 2
2 Any[Any[0, 1], Any[0, 1]]
[1.10862, 2.28941] [-1.89138, -0.710586]
[0.0, 1.0]


([0.0 0.0 … 0.0 0.0; 1.0 0.0 … 1.0 1.0], [-0.945534 -1.22468 … 0.494876 0.218439; -1.20745 0.964169 … -0.850412 -1.2125])

In [5]:
function risk_dom(n, params, a, x, e)
    
    #set up params
    delta = params[1:n]
    beta = params[n+1:2*n] 
    
    #setup payoffs
    u_lower = beta.*x  .+ e
    u_upper = u_lower[:] .+ delta
    
    #set up a
    a_return = zeros(n)
    
    #enumerate the mult eq
    entrants = 0
    for i=1:n
        if (length(a[i])>1)
            entrants = entrants +1
        end
    end
    
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
        if (length(a[i])>1) & (select == i)
            println("random select: ", select, " ", i)
            a_return[i] = 1 .- a[i][1]
        end
    end
    
    if length(a[1])>1
        println(select, " ", a)
        println(u_lower, " ", u_upper)
        println(a_return)
    end
    
    return a_return
end


risk_dom(3,  [-3,-3.5,-4, .6, .7, .8], [[0,1],[0,1],[1]], [1,2,3], [-1,-2,-3])

Any[Any[], Any[], Any[]]


In [ ]:
function pareto_opt(n, params, a, x, e; rho = -2)
    print("yo")
    a_return = zeros(n)
    lambda = exp(rho)/(1+exp(rho))
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
       
        if length(a[i])>1 & (select ==i)
            a_return[i] = 1 .- a[i][1]
        end
    end
    return a_return
end